## Introdução

No âmbito da UC Projeto, foi-nos sugerida a criação de um sistema para angariação e armazenamento de ficheiros de imagem num sistema de bases de dados vetorial, para posterior identificação de padrões de imagens.

No sentido de ir ao encontro do sugerido, decidimos criar um sistema de reconhecimento facial. Este sistema, permite-nos fornecer uma imagem como input, identifica as faces presentes na imagem e apresenta-nos como resultado as imagens das faces mais semelhantes ao input que foram encontradas na nossa base de dados.

Ao longo deste documento iremos detalhar quais as tecnologias utilizadas, as etapas do desenvolvimento deste sistema e particularidades da sua implemntação.

Por fim, faremos uma pequena demonstração do uso da aplicação.


### Contextualização e Motivação

O reconhecimento facial é um campo de investigação e desenvolvimento ativo há várias décadas, com inúmeras aplicações já implementadas no nosso dia a dia. Desde o desbloqueio de smartphones até aos sistemas de vigilância, a tecnologia tem demonstrado o seu potencial e importância crescente.

No entanto, existem vários desafios, sejam eles na identificação de características das faces ou no armazenamento das imagens numa base de dados. Lidar com variações na aparência facial devido a diferentes condições de iluminação, ângulos de visão, expressões faciais e oclusões (como óculos ou máscaras) continua a ser um problema complexo; as dificuldades que as bases de dados tradicionais têm no armazenamento de imagens é também um entrave: as imagens podem ser ficheiros grandes, e armazenar um grande volume delas diretamente na base de dados pode levar a um crescimento exponencial do tamanho da mesma, o que pode levar a um baixo desempenho aplicacional.

É neste contexto que a utilização de bases de dados vetoriais surge como uma alternativa promissora. Ao representar imagens como vetores num espaço de alta dimensão, onde a distância entre vetores reflete a similaridade entre as imagens, torna-se possível realizar pesquisas mais eficientes e encontrar correspondências mesmo quando as imagens apresentam variações significativas.

Este projeto vai de encontro às tendências atuais na área de reconhecimento de padrões e análise de imagem, explorando a cooperação entre técnicas de visão computacional e sistemas de armazenamento. 



## Tecnologias e Bibiliotecas

IDEIAS GERAIS:
- Linguagem utilizada: python
- Base de dados: Qdrant -> O que nos levou a escolher esta BD? Boa documentação, fácil configuração, possibilidade de usar localmente recorrendo a uma imagem docker
- Vision Transformers -> breve explicação de como funcionam com um exemplo mundano, quais escolhemos e porquê -- Fizemos a escolha baseada nos seguintes critérios: tarefa para o qual o modelo foi desenvolvido (Image Feature Extraction), tamanho do dataset com que foi treinado, empresas que os desenvolveram (google e facebook), resolução das imagens (em ambos 224x224) e dimensão do embedding gerado (ambos 768) - DATALOOP [1](https://dataloop.ai/library/model/)  e HUGGINGFACE [2](https://huggingface.co/models?pipeline_tag=image-feature-extraction)
- MTCNN -> biblioteca robusta para detecção facial, desenvolvida para detetar faces e os seus pontos de referência numa imagem, utilzando uma MCCN (multitask cascaded convolutional network)
- Outras bibliotecas para manipulação de dados numéricos e vetores e processamento de imagens


## Configuração Inicial

Antes de correr o código deste notebook, é necessário:
- Fazer download e guardar as imagens a inserir na base de dados numa pasta com nome "assets"
- Criar um ambiente virtual
- Instalar todos os packages necessários
- Garantir que temos a plataforma docker instalada e inicializar um container com Qdrant


```Bash
# Criar ambiente com conda
conda env create -n my_env python=3.11 pip
conda activate my_env

# Instalar packages
pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cpu

# Download da imagem de Qdrant
docker pull qdrant/qdrant
```

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from PIL import Image
import os

import qdrant.utils as qd
import models.models as mdl

In [ ]:

# Using containers
# client = QdrantClient(host="qdrant", port=6333)

# Local dev
client = QdrantClient(host="localhost", port=6333)

In [ ]:
collection = "image_collection"

# facebook/dino-vitb16 size: 768
# Vit Base Patch16 224 In21k size: 768
qd.create_collection(client, collection, 768, models.Distance.COSINE)

In [ ]:
current_directory = os.getcwd()
extensions = (".jpg", ".jpeg")

def get_images_data(path):
    data = []
    for file in os.listdir(f"{path}"):
        if file.endswith(extensions):
            file_name = file.split(".")[0].lower().replace("_"," ")
            name = "".join(c for c in file_name if c.isalpha() or c == ' ')[:-1] 
            img_data = {
                "name": name,
                "file_name": file,
                "img_obj": Image.open(os.path.join(f"{path}",file))
            }
            data.append(img_data)
    
    return data

data = get_images_data(f"{current_directory}/assets")

In [ ]:
data

In [ ]:
# Initialize models 
dino = "dino-vitb16"
device, processor, model, detector = mdl.init_models()


In [ ]:
# Create directory if needed
faces_path = "faces"
if not os.path.exists(faces_path):
    os.makedirs(faces_path)


destiny_path = f"{current_directory}/{faces_path}/"
for i, image_data in enumerate(data):
    mdl.process_image(image_data['img_obj'], 
                      detector,
                      destiny_path,
                      image_data['file_name'])

faces_data = get_images_data(f"{current_directory}/{faces_path}")

In [ ]:
faces_data, len(faces_data)

In [ ]:
# Generate embeddings
embeddings = mdl.gen_embeddings(faces_data, processor, device, model)

In [ ]:
embeddings[0]

In [ ]:
# Save embeddings to DB
for img_data in embeddings:
    qd.insert_image_embedding(client, collection, img_data)

In [ ]:
img_to_search, searched_faces = mdl.gen_embedding_img_to_search(f"{current_directory}/img_to_search/taskmaster.jpg", processor, device, model, detector)
img_to_search, searched_faces

In [ ]:
len(img_to_search), len(img_to_search[0])

In [ ]:
# Search top X similar results
top = 3
nearest_results = []
for img in img_to_search:
    nearest = qd.get_top_x_similar_images(client, collection, top, img)
    nearest_results.append(nearest)

nearest_results

In [ ]:
def print_results(results):
    msg = f"{len(results)} faces were found in the provided image" if len(results) > 1 else f"{len(results)} face was found in the provided image"
    print(msg)
    for j, result in enumerate(results):
        # print(f"{j}: {result.points}")
        points = result.points
        print(f"Face {j+1}:")
        searched_faces[j].show()
        for i in range(len(points)):
            name = points[i].payload['name']
            score = points[i].score
            print(f"Result #{i+1}: {name} was diagnosed with {score * 100} confidence")
            print(f"This image score was {score}")
            Image.open(f"faces/{points[i].payload['file_name']}").show()
            print("-" * 50)
            print()

print_results(nearest_results)

In [ ]:
img_data_to_add = [{
   'img_path': 'img_to_add/João_Baião_1.jpg',
   'payload': {
      'name': 'João Baião',
      'file_name': 'João_Baião_1.jpg'
   }
},
{
   'img_path': 'img_to_add/João_Baião_2.jpg',
   'payload': {
      'name': 'João Baião',
      'file_name': 'João_Baião_2.jpg'
   }
},
{
   'img_path': 'img_to_add/João_Baião_3.jpg',
   'payload': {
      'name': 'João Baião',
      'file_name': 'João_Baião_3.jpg'
   }
}]

def add_img_to_db(data_to_add: list[dict]):
   img_data = []
   for data in data_to_add:
      img_obj = Image.open(data['img_path'])
      img_data.append({
         'name': data['payload']['name'],
         'file_name': data['payload']['file_name'],
         'img_obj': img_obj
      })
   embeddings = mdl.gen_embeddings(img_data, processor, device, model)
   for img in embeddings:
      qd.insert_image_embedding(client, collection, img)
   

add_img_to_db(img_data_to_add)


In [ ]:
res = qd.get_points_from_collection(client, collection, 5)
res

In [ ]:
id = ''
qd.delete_points_from_collection(client, collection, [id])

In [ ]:
!streamlit run search_image.py